In [1]:
import asyncio
from time import sleep
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

In [10]:
!pip show langchain-mcp-adapters

Name: langchain-mcp-adapters
Version: 0.0.10
Summary: Make Anthropic Model Context Protocol (MCP) tools compatible with LangChain and LangGraph agents.
Home-page: 
Author: 
Author-email: Vadym Barda <19161700+vbarda@users.noreply.github.com>
License: 
Location: /home/suman/.local/lib/python3.10/site-packages
Requires: langchain-core, mcp
Required-by: 


In [2]:
!pip show langchain

Name: langchain
Version: 0.3.24
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/suman/.local/lib/python3.10/site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [3]:
import urllib.parse
base_url_1 = "http://localhost:5555/devmode/exampleApplication/privkey/session1/sse"
params_1 = {
    "waitForAgents": 1,
    "agentId": "test_agent",
    "agentDescription": "You are the first test agent."
}
query_string = urllib.parse.urlencode(params_1)
MCP_SERVER_URL_1 = f"{base_url_1}?{query_string}"
async def connect_client():
    global mcp_client_1
    mcp_client_1 = MultiServerMCPClient(
        connections={
            "coral": {
                "transport": "sse",
                "url": MCP_SERVER_URL_1,
                "timeout": 5,
                "sse_read_timeout": 300,
            }
        }
    )
    await mcp_client_1.__aenter__()
    print(f"Connected to MCP server as test_agent1 at {MCP_SERVER_URL_1}")

await connect_client()


Connected to MCP server as test_agent1 at http://localhost:5555/devmode/exampleApplication/privkey/session1/sse?waitForAgents=1&agentId=test_agent&agentDescription=You+are+the+first+test+agent.


Error in sse_reader: 


In [4]:
import urllib.parse
from langchain_mcp_adapters.client import MultiServerMCPClient

base_url_2 = "http://localhost:5555/devmode/exampleApplication/privkey/session1/sse"
params_2 = {
    "waitForAgents": 1,
    "agentId": "test_agent2",
    "agentDescription": "You are the second test agent."
}
query_string_2 = urllib.parse.urlencode(params_2)
MCP_SERVER_URL_2 = f"{base_url_2}?{query_string_2}"

async def connect_second_agent():
    global mcp_client_2
    mcp_client_2 = MultiServerMCPClient(
        connections={
            "coral": {
                "transport": "sse",
                "url": MCP_SERVER_URL_2,
                "timeout": 5,
                "sse_read_timeout": 300,
            }
        }
    )
    await mcp_client_2.__aenter__()
    print(f"Connected to MCP server as test_agent2 at {MCP_SERVER_URL_2}")

await connect_second_agent()


Connected to MCP server as test_agent2 at http://localhost:5555/devmode/exampleApplication/privkey/session1/sse?waitForAgents=1&agentId=test_agent2&agentDescription=You+are+the+second+test+agent.


Error in sse_reader: 


In [5]:
async def get_tools():
    global tools_by_name1, tool_names1
    try:
        tools = mcp_client_1.get_tools()
        tool_names1 = [tool.name for tool in tools]
        tools_by_name1 = {tool.name: tool for tool in tools}
        print(f"get_tools result: Available tools for test_agent: {tool_names1}")
    except Exception as e:
        print(f"get_tools error: Failed to retrieve tools: {str(e)}")

await get_tools()

get_tools result: Available tools for test_agent: ['list_agents', 'create_thread', 'add_participant', 'remove_participant', 'close_thread', 'send_message', 'wait_for_mentions']


In [6]:
async def get_tools_agent2():
    global tools_by_name2, tool_names2
    try:
        tools = mcp_client_2.get_tools()
        tool_names2 = [tool.name for tool in tools]
        tools_by_name2 = {tool.name: tool for tool in tools}
        print(f"get_tools_agent2 result: Available tools for test_agent2: {tool_names2}")
    except Exception as e:
        print(f"get_tools_agent2 error: Failed to retrieve tools for test_agent2: {str(e)}")

await get_tools_agent2()

get_tools_agent2 result: Available tools for test_agent2: ['list_agents', 'create_thread', 'add_participant', 'remove_participant', 'close_thread', 'send_message', 'wait_for_mentions']


In [7]:
async def list_agents():
    try:
        result = await tools_by_name1["list_agents"].ainvoke({
            "includeDetails": True
        })
        print("list_agents result:")
        lines = result.strip().split("\n")
        print(lines[0])  # Header
        for line in lines[1:]:
            parts = [p.strip() for p in line.split(",")]
            for part in parts:
                print(f"  - {part}")
    except KeyError:
        print("list_agents error: Tool 'list_agents' not found")
    except Exception as e:
        print(f"list_agents error: Failed to list agents: {str(e)}")

await list_agents()

list_agents result:
Registered Agents (3):
  - ID: user_interaction_agent
  - 
  - Description: An agent that takes the user's input and interacts with other agents to fulfill the request
  - ID: test_agent2
  - 
  - Description: You are the second test agent.
  - ID: test_agent
  - 
  - Description: You are the first test agent.


In [8]:
async def create_thread():
    global thread_id
    try:
        result = await tools_by_name1["create_thread"].ainvoke({
            "threadName": "test_thread",
            "participantIds": ["test_agent2"]
        })
        # Parse thread ID from result (format: "Thread created successfully:\nID: <thread_id>\n...")
        lines = result.strip().split("\n")
        for line in lines:
            if line.startswith("ID:"):
                thread_id = line.split(":", 1)[1].strip()
                break
        else:
            thread_id = None
            print("create_thread warning: Thread ID not found in result")
        print(f"create_thread result: {result}")
    except KeyError:
        print("create_thread error: Tool 'create_thread' not found")
    except Exception as e:
        print(f"create_thread error: Failed to create thread: {str(e)}")

await create_thread()

create_thread result: Thread created successfully:
ID: 6baa2eb9-05e6-451e-b3d9-40838b5d78c0
Name: test_thread
Creator: test_agent
Participants: test_agent2, test_agent


In [9]:
async def send_message():
    try:
        if thread_id is None:
            print("send_message error: No thread ID available")
            return
        result = await tools_by_name1["send_message"].ainvoke({
            "threadId": thread_id,
            "content": "Hello, this is a message from Agent1!",  # Updated to match received message
            "mentions": ["test_agent2"]
        })
        print(f"send_message result: {result}")
    except KeyError:
        print("send_message error: Tool 'send_message' not found")
    except Exception as e:
        print(f"send_message error: Failed to send message: {str(e)}")

await send_message()

send_message result: Message sent successfully:
ID: fbae34c6-d585-4830-a24b-5bd5f72adf53
Thread: 6baa2eb9-05e6-451e-b3d9-40838b5d78c0
Sender: test_agent
Content: Hello, this is a message from Agent1!
Mentions: test_agent2


In [12]:
async def wait_for_mentions_agent2():
    try:
        result = await tools_by_name2["wait_for_mentions"].ainvoke({
            "timeoutMs": 8000
        })
        print(f"wait_for_mentions_agent2 result: {result}")
    except KeyError:
        print("wait_for_mentions_agent2 error: Tool 'wait_for_mentions' not found")
    except Exception as e:
        print(f"wait_for_mentions_agent2 error: Failed to wait for mentions: {str(e)}")

await wait_for_mentions_agent2()

wait_for_mentions_agent2 error: Failed to wait for mentions: name 'tools_by_name2' is not defined


In [13]:
async def send_another_message():
    try:
        if thread_id is None:
            print("send_another_message error: No thread ID available")
            return
        result = await tools_by_name2["send_message"].ainvoke({
            "threadId": thread_id,
            "content": "Hi, I received your message!",
            "mentions": ["test_agent"]
        })
        print(f"send_another_message result: {result}")
    except KeyError:
        print("send_another_message error: Tool 'send_message' not found")
    except Exception as e:
        print(f"send_another_message error: Failed to send message: {str(e)}")

await send_another_message()

send_another_message error: Failed to send message: name 'tools_by_name2' is not defined


In [14]:
async def wait_for_mentions_agent2():
    try:
        result = await tools_by_name1["wait_for_mentions"].ainvoke({"timeoutMs": 8000})
        print(f"wait_for_mentions_agent2 result: {result}")
        return result
    except KeyError:
        print("wait_for_mentions_agent2 error: Tool 'wait_for_mentions' not found")
        return None
    except Exception as e:
        print(f"wait_for_mentions_agent2 error: Failed to wait for mentions: {str(e)}")
    return None

await wait_for_mentions_agent2()

Error in post_writer: 
Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


wait_for_mentions_agent2 result: No new messages received within the timeout period


'No new messages received within the timeout period'

In [27]:
# for this test we have already added a participant to the thread, hence it will fail
# async def add_participant():
#     result = await tools_by_name1["add_participant"].ainvoke({
#         "threadId": "753f0f70-972d-45dd-86b6-372ec49c6767",
#         "participantId": "test_agent3"
#     })
#     print(f"Add participant result: {result}")

# await add_participant()

In [28]:
# async def remove_participant():
#     result = await tools_by_name1["remove_participant"].ainvoke({
#         "threadId": "753f0f70-972d-45dd-86b6-372ec49c6767",
#         "participantId": "test_agent2"
#     })
#     print(f"Remove participant result: {result}")

# await remove_participant()

In [29]:
# async def close_thread():
#     result = await tools_by_name1["close_thread"].ainvoke({
#         "threadId": "753f0f70-972d-45dd-86b6-372ec49c6767",
#         "summary": "Discussion completed between test_agent and test_agent2."
#     })
#     print(f"Close thread result: {result}")

# await close_thread()